# VacationPy
----

Planning future vacations

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Importing API key
from api_keys import g_key

### Store Part I results into DataFrame
* Loading the csv exported in Part I to a DataFrame

In [2]:
#Reading and opening Weatherpy DataFrame
data_file = os.path.join("output_data","cities.csv")
weather_df= pd.read_csv(data_file)
weather_df.head()


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Al Bardīyah,LY,31.76,25.09,77.05,40,0,13.31,1592272020
1,Avarua,CK,-21.21,-159.78,73.40,56,56,6.93,1592272020
2,Cape Town,ZA,-33.93,18.42,46.99,93,0,2.24,1592271902
3,Guilin,CN,25.28,110.29,77.00,94,75,2.24,1592272021
4,Ponta do Sol,PT,32.67,-17.10,69.80,88,75,3.36,1592272021


### Humidity Heatmap

In [3]:
# Accessing maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Storing latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Storing humidity in a variable that will be used as the "Weight" parameter for the heat map
humidity_values = weather_df["Humidity"].astype(float)

# Plotting Heatmap
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Defining maps parameters
fig = gmaps.figure(zoom_level=1.4, center= (51.4934,0.0098), layout=figure_layout)

# Creating heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_values, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Adding the  layer
fig.add_layer(heat_layer)

# Displaying the  figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Filtering and adjusting the source dataframe to find desired weather conditions for future vacations

In [5]:
# Filtering ideal weather conditions, and creting a new dataframe
hotel_df= weather_df.loc[((weather_df["Max Temp"]>80) & (weather_df["Max Temp"]<=90)) & (weather_df["Wind Speed"]<10) \
                              & (weather_df["Humidity"]<50) & (weather_df["Cloudiness"]<5)].copy()
                                                                                                     
hotel_df

# Null values were removed in the previous analysis.

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
148,Harrison,US,30.41,-89.04,86.00,48,1,3.36,1592271816
251,Frontera,MX,26.93,-101.45,87.01,31,1,8.01,1592272058
322,Farah,AF,32.50,62.50,84.85,13,0,4.18,1592272069
399,Makkah al Mukarramah,SA,21.43,39.83,89.08,21,0,5.10,1592272081
418,Belaya Gora,RU,68.53,146.42,84.00,22,0,7.27,1592272085


### Hotel Map

In [6]:
# Adding empty columns to hotel_df
hotel_df["Hotel Name"]= ""
hotel_df["Lat"]= ""
hotel_df["Lng"]= ""
hotel_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Lat,Lng
148,Harrison,US,30.41,-89.04,86.00,48,1,3.36,1592271816,,,
251,Frontera,MX,26.93,-101.45,87.01,31,1,8.01,1592272058,,,
322,Farah,AF,32.50,62.50,84.85,13,0,4.18,1592272069,,,
399,Makkah al Mukarramah,SA,21.43,39.83,89.08,21,0,5.10,1592272081,,,
418,Belaya Gora,RU,68.53,146.42,84.00,22,0,7.27,1592272085,,,


In [7]:
# Defining API url parameters
params = {
    "radius": 5000,
    "type":"lodging",
    "keyword":"hotel",
    "key": g_key
}

# Looping through each dataframe index and rows
for index, row in hotel_df.iterrows():
    # getting lat, lng from hotel_df
    lat = row["Latitude"]
    lng = row["Longitude"]
# changing location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Defining base Google Place API url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # making the request and getting the json response
    hotel_response = requests.get(base_url, params=params).json()
# Creating an exception to avoid keyerrors    
    try:
    
        print(f"Nice! we have found {hotel_response['results'][0]['name']} in {row['City']}")
        
        hotel_df.loc[index,'Hotel Name'] = hotel_response['results'][0]['name']
        hotel_df.loc[index,'Lat'] = hotel_response['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index,'Lng'] = hotel_response['results'][0]['geometry']['location']['lng']
        
    except:
        #Printing an explanation message
        print(f"No Hotel Found in {row['City']}")
        # For those hotels that are not founded, th program must put np.NaN values over those cells
        hotel_df.loc[index,'Hotel Name'] = np.NaN
        hotel_df.loc[index,'Lat']= np.NaN
        hotel_df.loc[index,'Lng'] = np.NaN
        pass
    
# Dropping np.NaN values    
hotel_df.dropna(inplace=True)
        
    
# Organizing and printing hotel_df 

hotel_df= hotel_df[["City","Country","Lat","Lng","Hotel Name"]]
hotel_df

Nice! we have found Best Western Seaway Inn in Harrison
Nice! we have found Hotel Olímpia in Frontera
No Hotel Found in Farah
Nice! we have found Pullman Zamzam Makkah in Makkah al Mukarramah
No Hotel Found in Belaya Gora


,City,Country,Lat,Lng,Hotel Name
148,Harrison,US,30.426,-89.0937,Best Western Seaway Inn
251,Frontera,MX,26.9017,-101.416,Hotel Olímpia
399,Makkah al Mukarramah,SA,21.4193,39.8247,Pullman Zamzam Makkah


In [8]:
# Defining info_box_template
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Storing info_box_content in hotel_info variable
# A list comprehension loop to store hotel info values
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Adding marker layer ontop of heat map
markers= gmaps.marker_layer(locations,info_box_content= hotel_info)
fig.add_layer(markers)

# Displaying figure

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…